# Automated Praat formant measurements

This notebook illustrates how to run the Praat `To Formant... (Burg)` formant tracker on a corpus of audio files and collect the results in `.csv` files of time-aligned formant measurements. The [parselmouth library](https://parselmouth.readthedocs.io/en/stable/) provides access to Praat's speech processing routines in Python.

The [X-Ray Microbeam database](https://github.com/rsprouse/xray_microbeam_database) will be used for the illustration. This database contains audio and associated articulatory data from 48 speakers. Aligned word and phone labels for the audio are available separately.

## The workflow

The steps for processing files is as follows:

1. Read the input data filenames (`*.wav`) into a DataFrame.
1. Run the formant analysis for each input file that is missing an output and cache the results in a `.csv` file.

In [1]:
import os, re
import subprocess
from pathlib import Path
import pandas as pd
from parselmouth import Sound
from phonlab.utils import dir2df
from phonlab.tidypraat import formant2df

First we define file locations. The name of the directory containing the microbeam database files is stored in `xraydir`, and `cachedir` is where output `.csv` files will be saved.

In [2]:
xraydir = Path.home() / 'xray_microbeam_database'
cachedir = Path.home() / 'xray_formants_praat'

## Combine speaker metadata and analysis parameters

Before loading input filenames we load speaker metadata from the `speaker_demographics1.csv` file. Use the `read_csv()` function to read the `subject` and `sex` columns from it. The latter is also renamed to `spkr`.

In [3]:
md = pd.read_csv(
    xraydir / 'speaker_demographics1.csv',
    usecols=['subject', 'sex']
).rename({'sex': 'spkr'}, axis='columns')
md.head()

,subject,spkr
0,JW05,F
1,JW06,M
2,JW07,M
3,JW08,M
4,JW09,F


Next we define lpc analysis parameters based on speaker type.

In [4]:
fmtparams = pd.DataFrame({
    'spkr':    ['F', 'M',  'child'],
    'ceiling': [5500, 5000, 8000]
})
fmtparams

,ceiling,spkr
0,5500,F
1,5000,M
2,8000,child


Merge the subject metadata with the analysis parameters based on speaker type, which results in speaker-dependent values for `ceiling` that will be used as a formant analysis parameter.

In [5]:
md = md.merge(fmtparams, on='spkr', how='left')
md.head()

,subject,spkr,ceiling
0,JW05,F,5500
1,JW06,M,5000
2,JW07,M,5000
3,JW08,M,5000
4,JW09,F,5500


# Load xray filenames

Load the .wav files in the xray database. For demonstration purposes we select only a subset of `.wav` files for subjects `JW6*` for analysis.

In [6]:
dirpat = '^JW6'
#dirpat = '^JW' # use this instead for all subject directories
fnpat = '^tp(?P<task>00\d)\.wav$'
#fnpat = '^t[ap](?P<task>\d+)\.wav$' # use this instead for all .wav files
wavdf = dir2df(
    xraydir,
    dirpat=dirpat,
    fnpat=fnpat,
    addcols=['dirname', 'barename', 'ext']
)
wavdf

,dirname,relpath,fname,barename,ext,task
0,/Users/oski/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001
1,/Users/oski/xray_microbeam_database,JW60,tp002.wav,tp002,.wav,002
2,/Users/oski/xray_microbeam_database,JW60,tp003.wav,tp003,.wav,003
3,/Users/oski/xray_microbeam_database,JW60,tp004.wav,tp004,.wav,004
4,/Users/oski/xray_microbeam_database,JW60,tp005.wav,tp005,.wav,005
5,/Users/oski/xray_microbeam_database,JW60,tp006.wav,tp006,.wav,006
6,/Users/oski/xray_microbeam_database,JW60,tp007.wav,tp007,.wav,007
7,/Users/oski/xray_microbeam_database,JW60,tp008.wav,tp008,.wav,008
8,/Users/oski/xray_microbeam_database,JW60,tp009.wav,tp009,.wav,009
9,/Users/oski/xray_microbeam_database,JW61,tp001.wav,tp001,.wav,001


## Merge filenames with speaker metadata.

The filenames to be analyzed are stored in `wavdf`. Next we merge it with the metadata in `md`. After merging, `wavdf` contains rows with target filenames and the formant analysis parameter `ceiling` to be used on that file.

In [7]:
wavdf = wavdf.merge(
    md,
    left_on='relpath',
    right_on='subject',
    how='left'
)
wavdf[wavdf['barename'] == 'tp001']

,dirname,relpath,fname,barename,ext,task,subject,spkr,ceiling
0,/Users/oski/xray_microbeam_database,JW60,tp001.wav,tp001,.wav,001,JW60,F,5500
9,/Users/oski/xray_microbeam_database,JW61,tp001.wav,tp001,.wav,001,JW61,M,5000
18,/Users/oski/xray_microbeam_database,JW62,tp001.wav,tp001,.wav,001,JW62,F,5500
27,/Users/oski/xray_microbeam_database,JW63,tp001.wav,tp001,.wav,001,JW63,M,5000


## Do formant analysis

Define some variables to be used to control formant analysis.

The global formant analysis parameters are defined in `globalparams`. See the [Praat manual's description of the 'To formant (burg)...' function](https://www.fon.hum.uva.nl/praat/manual/Sound__To_Formant__burg____.html) for their meanings and usage. The `maximum_formant` param is commented out because its value will be assigned per-speaker and does not have a global value.

Use `overwrite` to allow/prevent re-running analysis on `.wav` files that have previously been processed. Use `errors` to either immediately stop processing when an error occurs or to write a warning message and continue processing.

In [8]:
globalparams = {  # Uncomment and change formant analysis params as desired
    'time_step': None,  # None == 'auto'
    'max_number_of_formants': 5,
#    'maximum_formant': 5500,
    'window_length': 0.025,
    'pre_emphasis_from': 50
}

overwrite = False # If False, do not perform formant analysis if cached file
                  # already exists. If True, do analysis and overwrite existing file.
errors = 'raise'  # `raise` stops processing on error;
                  # change to 'ignore' to only print a warning message.

Loop over the `.wav` files and do formant analysis of each and cache the results. Re-running the loop will not redo analysis of previously-processed files unless `overwrite` is `True`.

Note that the cache filename includes the values of two of the analysis parameters. You can experiment with different analysis parameters and explore the results in their respective cache files.



In [9]:
for row in wavdf.itertuples():
    wavpath = xraydir / row.relpath / row.fname  # Full path to .wav file
    # Name of cachefile
    cachename = f"{row.barename}.{row.ceiling}ceil.{globalparams['max_number_of_formants']}formant.csv"
    cachepath = cachedir / row.relpath / cachename   # Full path to cachefile
    if overwrite is True or not cachepath.exists():
        try:
            snd = Sound(str(wavpath))         # Parselmouth requires a str
            formant = snd.to_formant_burg(
                maximum_formant=row.ceiling,  # Per-speaker param
                **globalparams                # Global params
            )
            fdf = formant2df(
                formant,
                num=globalparams['max_number_of_formants'],
                include_bw=True,
                tcol='t1'
            )
            cachepath.parent.mkdir(parents=True, exist_ok=True)  # Make sure parent dir exists
            # Save formant measurements as .csv
            fdf.to_csv(cachepath, index=False)
            print(f'Wrote formant results to {row.relpath}/{cachename}')
            # Uncomment if you want to open formant object in Praat for drawing
            # formant.save_as_binary_file(str(cachepath.with_suffix('.pformant')))
        except Exception as e:
            if errors == 'ignore':
                sys.stderr.write(f'Error processing {wavpath}\n\n{e}')
            else:
                raise e

Wrote formant results to JW60/tp001.5500ceil.5formant.csv
Wrote formant results to JW60/tp002.5500ceil.5formant.csv
Wrote formant results to JW60/tp003.5500ceil.5formant.csv
Wrote formant results to JW60/tp004.5500ceil.5formant.csv
Wrote formant results to JW60/tp005.5500ceil.5formant.csv
Wrote formant results to JW60/tp006.5500ceil.5formant.csv
Wrote formant results to JW60/tp007.5500ceil.5formant.csv
Wrote formant results to JW60/tp008.5500ceil.5formant.csv
Wrote formant results to JW60/tp009.5500ceil.5formant.csv
Wrote formant results to JW61/tp001.5000ceil.5formant.csv
Wrote formant results to JW61/tp002.5000ceil.5formant.csv
Wrote formant results to JW61/tp003.5000ceil.5formant.csv
Wrote formant results to JW61/tp004.5000ceil.5formant.csv
Wrote formant results to JW61/tp005.5000ceil.5formant.csv
Wrote formant results to JW61/tp006.5000ceil.5formant.csv
Wrote formant results to JW61/tp007.5000ceil.5formant.csv
Wrote formant results to JW61/tp008.5000ceil.5formant.csv
Wrote formant 